<a href="https://colab.research.google.com/github/invi-bhagyesh/study-GPT/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/invi-bhagyesh/study-GPT.git

Cloning into 'study-GPT'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 109 (delta 28), reused 98 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (109/109), 4.40 MiB | 4.85 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd study-GPT

/content/study-GPT


In [7]:
# Optional: install wandb if you want
# !pip install wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
import math
from tqdm import tqdm

# Optional
# import wandb

In [8]:
from model.gpt2 import GPT2


In [9]:
# For GPT-2
from transformers import GPT2Config
from model.gpt2 import GPT2  # assuming you have your custom GPT2 implementation

# Create config
config = GPT2Config(
    n_layer=6,    # number of transformer layers
    n_head=8,     # number of attention heads
    n_embd=512    # embedding dimension
)

# Initialize model
model = GPT2(config)

# 1. Basic parameter count
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Count parameters
total_params = count_parameters(model)
print(f"Total trainable parameters: {total_params:,}")

Total trainable parameters: 45,171,200


In [ ]:
"""
from transformers import GPT2Tokenizer
from data.dataset import TextDataset
from train import train_gpt2

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
texts = dataset["train"]["text"]  # Just get the plain text list


# Create dataset
dataset = TextDataset(texts, tokenizer)
val_dataset = TextDataset(dataset["validation"]["text"], tokenizer)

# Train the model
model = train_gpt2(
    train_dataset=dataset,
    val_dataset=val_dataset,
    n_epochs=3,
    batch_size=8,
    learning_rate=3e-4
)

# Save the model
#torch.save(model.state_dict(), 'gpt2_model.pt')
"""

In [10]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [11]:
from transformers import GPT2Tokenizer
from datasets import load_dataset
from train import train_gpt2

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Tokenize
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=1024)

tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

tokenized_dataset.save_to_disk("data/tokenized_wikitext2")

train_dataset = tokenized_dataset["train"]
val_dataset = tokenized_dataset["validation"]



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4358 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/36718 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [12]:
tokenized_dataset["train"]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 36718
})

In [13]:
total_tokens = sum([len(tokenizer(x['text'])['input_ids']) for x in dataset['train']])
print(total_tokens)

2391884


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from model.config import GPT2Config
from model.gpt2 import GPT2
from transformers import GPT2Tokenizer
from tqdm.notebook import tqdm   # NOTE: notebook-friendly tqdm
import wandb

In [ ]:
!wandb login

In [ ]:
from train import train_gpt2

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

train_loader = DataLoader(tokenized_dataset["train"], batch_size=8, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(tokenized_dataset["validation"], batch_size=8, collate_fn=data_collator)


In [ ]:
train_gpt2(train_loader, val_loader, n_epochs=5)
# Save the model after training
torch.save(model.state_dict(), "data/gpt2_pretrained.pth")
print("Model saved to data/gpt2_pretrained.pth")

In [ ]:
# Train
model = train_gpt2(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    n_epochs=3,
    batch_size=8,
    learning_rate=3e-4
)